In [1]:
!pip install streamlit ngrok scikit-learn matplotlib seaborn pandas numpy tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 502.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [2]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model

# ---------------------- Load Model ----------------------
@st.cache_resource
def load_trained_model():
    model = load_model('model.h5')
    return model

# ---------------------- Function to Preprocess Data ----------------------
def preprocess_data(df):
    df.columns = df.columns.str.strip().str.lower()  # Clean column names
    df = df.select_dtypes(include=[np.number])  # Drop non-numeric columns
    df.fillna(df.mean(), inplace=True)  # Fill missing values with mean
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df)
    return scaled_data, scaler

# ---------------------- PCA Function ----------------------
def perform_pca(data):
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(data)
    explained_variance = pca.explained_variance_ratio_
    return pca_result, explained_variance

# ---------------------- Prediction Function ----------------------
def predict(model, data):
    predictions = model.predict(data)
    return predictions

# ---------------------- Streamlit UI ----------------------
def main():
    st.title("🌽 Hyperspectral Imaging Data Analysis")

    st.sidebar.title("🔍 Options")

    # File Upload
    uploaded_file = st.sidebar.file_uploader("Upload CSV Data", type=["csv"])

    if uploaded_file:
        # Load data
        df = pd.read_csv(uploaded_file)
        st.write("### Raw Data")
        st.dataframe(df.head())

        # Preprocessing
        data, scaler = preprocess_data(df)
        st.write("### Preprocessed Data")
        st.dataframe(pd.DataFrame(data, columns=df.columns))

        # PCA
        pca_result, explained_variance = perform_pca(data)
        st.write(f"### PCA Explained Variance: {explained_variance}")

        # PCA Scatter Plot
        fig, ax = plt.subplots()
        scatter = ax.scatter(pca_result[:, 0], pca_result[:, 1], s=20, alpha=0.7, cmap='viridis')
        ax.set_title('PCA Result')
        ax.set_xlabel('Principal Component 1')
        ax.set_ylabel('Principal Component 2')
        plt.colorbar(scatter)
        st.pyplot(fig)

        # Load Model
        model = load_trained_model()

        # Prediction
        if st.sidebar.button("Predict DON Concentration"):
            predictions = predict(model, data)
            df['Predicted DON Concentration'] = predictions
            st.write("### Predictions")
            st.dataframe(df[['Predicted DON Concentration']])

            # Scatter Plot of Actual vs Predicted (if actual available)
            if 'vomitoxin_ppb' in df.columns:
                fig, ax = plt.subplots()
                ax.scatter(df['vomitoxin_ppb'], df['Predicted DON Concentration'], alpha=0.7, edgecolors=(0, 0, 0))
                ax.set_title('Actual vs Predicted DON Concentration')
                ax.set_xlabel('Actual DON Concentration')
                ax.set_ylabel('Predicted DON Concentration')
                ax.plot([df['vomitoxin_ppb'].min(), df['vomitoxin_ppb'].max()],
                        [df['vomitoxin_ppb'].min(), df['vomitoxin_ppb'].max()],
                        color='red')
                st.pyplot(fig)

            # Download Predictions
            st.sidebar.download_button(
                label="📥 Download Predictions",
                data=df.to_csv(index=False).encode('utf-8'),
                file_name='predictions.csv',
                mime='text/csv'
            )

# Run the Streamlit app
if __name__ == "__main__":
    main()


Writing app.py


In [4]:
from google.colab import files
uploaded = files.upload()


Saving model.h5 to model.h5


In [5]:
!pip install pyngrok


In [ ]:
from pyngrok import ngrok

# Start Streamlit
!streamlit run app.py &

# Open Ngrok tunnel to the Streamlit port (8501)
public_url = ngrok.connect(8501)
print(f"Streamlit App is running at: {public_url}")





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8503
  Network URL: http://172.28.0.12:8503
  External URL: http://104.196.71.34:8503

